In [12]:
!pip install langchain openai

## LLM

In [13]:
from getpass import getpass

OPENAI_API_KEY = getpass()

import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

··········


In [19]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "Given the following SQL tables, your job is to write queries given a user’s request.\n  Table orders {\n  id int [primary key] \n  user_id int [not null, unique] \n  status varchar \n  created_at varchar \n } \n Table order_items  { \n  order_id int \n  product_id int \n  quantity int \n }\n Table products  { \n  id int [primary key] \n  name varchar \n  merchant_id int [not null] \n  price int \n  status varchar \n  created_at varchar \n  category_id int \n} \n Table users { \n  id int [primary key] \n  full_name varchar \n  email varchar [unique] \n  gender varchar \n  date_of_birth varchar \n  created_at varchar \n  country_code int \n }"
    },
    {
      "role": "user",
      "content": "Write a SQL query which gives List of products where current month sales is greater than as of in the same month of each last 3 years"
    }
  ],
  temperature=0.7,
  top_p=1
)
print(response.choices[0].message.content)

To get the list of products where the current month sales is greater than the sales of the same month in each of the last 3 years, you can use the following SQL query:

```sql
SELECT p.id, p.name, SUM(oi.quantity) AS current_month_sales
FROM products p
JOIN order_items oi ON p.id = oi.product_id
JOIN orders o ON oi.order_id = o.id
WHERE o.status = 'completed'
  AND YEAR(o.created_at) >= YEAR(CURRENT_DATE()) - 3
  AND MONTH(o.created_at) = MONTH(CURRENT_DATE())
GROUP BY p.id, p.name
HAVING current_month_sales > (
  SELECT SUM(oi.quantity)
  FROM products p2
  JOIN order_items oi ON p2.id = oi.product_id
  JOIN orders o ON oi.order_id = o.id
  WHERE o.status = 'completed'
    AND YEAR(o.created_at) = YEAR(CURRENT_DATE()) - 1
    AND MONTH(o.created_at) = MONTH(CURRENT_DATE())
)
AND current_month_sales > (
  SELECT SUM(oi.quantity)
  FROM products p3
  JOIN order_items oi ON p3.id = oi.product_id
  JOIN orders o ON oi.order_id = o.id
  WHERE o.status = 'completed'
    AND YEAR(o.created_a

In [15]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "Given the following SQL tables, your job is to write queries given a user’s request.\n  Table orders {\n  id int [primary key] \n  user_id int [not null, unique] \n  status varchar \n  created_at varchar \n } \n Table order_items  { \n  order_id int \n  product_id int \n  quantity int \n }\n Table products  { \n  id int [primary key] \n  name varchar \n  merchant_id int [not null] \n  price int \n  status varchar \n  created_at varchar \n  category_id int \n} \n Table users { \n  id int [primary key] \n  full_name varchar \n  email varchar [unique] \n  gender varchar \n  date_of_birth varchar \n  created_at varchar \n  country_code int \n }"
    },
    {
      "role": "user",
      "content": "Write a SQL query which gives List of customers where current month sales figures < 50% as of in the month which is before 3 months the current month"
    }
  ],
  temperature=0.0,
  top_p=1
)
print(response.choices[0].message.content)

To get the list of customers where the current month sales figures are less than 50% compared to the sales figures of the month which is three months before the current month, you can use the following SQL query:

```sql
SELECT u.full_name, u.email
FROM users u
JOIN orders o ON u.id = o.user_id
JOIN order_items oi ON o.id = oi.order_id
JOIN products p ON oi.product_id = p.id
WHERE o.status = 'completed'
  AND DATE_FORMAT(o.created_at, '%Y-%m') = DATE_FORMAT(CURRENT_DATE, '%Y-%m')
  AND p.price * oi.quantity < (
    SELECT 0.5 * SUM(p.price * oi.quantity)
    FROM orders o
    JOIN order_items oi ON o.id = oi.order_id
    JOIN products p ON oi.product_id = p.id
    WHERE o.status = 'completed'
      AND DATE_FORMAT(o.created_at, '%Y-%m') = DATE_FORMAT(DATE_SUB(CURRENT_DATE, INTERVAL 3 MONTH), '%Y-%m')
  )
GROUP BY u.id, u.full_name, u.email;
```

This query joins the `users`, `orders`, `order_items`, and `products` tables. It filters the orders based on the status being 'completed' and 

In [16]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "Given the following SQL tables, your job is to write queries given a user’s request.\n  Table orders {\n  id int [primary key] \n  user_id int [not null, unique] \n  status varchar \n  created_at varchar \n } \n Table order_items  { \n  order_id int \n  product_id int \n  quantity int \n }\n Table products  { \n  id int [primary key] \n  name varchar \n  merchant_id int [not null] \n  price int \n  status varchar \n  created_at varchar \n  category_id int \n} \n Table users { \n  id int [primary key] \n  full_name varchar \n  email varchar [unique] \n  gender varchar \n  date_of_birth varchar \n  created_at varchar \n  country_code int \n }"
    },
    {
      "role": "user",
      "content": "Write a SQL query which gives List of customers where current month sales figures < 50% as of in the same month of the last year"
    }
  ],
  temperature=0.0,
  top_p=1
)
print(response.choices[0].message.content)

To get the list of customers where the current month sales figures are less than 50% compared to the same month of the last year, you can use the following SQL query:

```sql
SELECT u.full_name, u.email
FROM users u
JOIN orders o ON u.id = o.user_id
JOIN order_items oi ON o.id = oi.order_id
JOIN products p ON oi.product_id = p.id
WHERE o.status = 'completed'
  AND MONTH(o.created_at) = MONTH(CURRENT_DATE())
  AND YEAR(o.created_at) = YEAR(CURRENT_DATE())
  AND p.price * oi.quantity < (
    SELECT SUM(p.price * oi.quantity)
    FROM orders o
    JOIN order_items oi ON o.id = oi.order_id
    JOIN products p ON oi.product_id = p.id
    WHERE o.status = 'completed'
      AND MONTH(o.created_at) = MONTH(CURRENT_DATE() - INTERVAL 1 YEAR)
      AND YEAR(o.created_at) = YEAR(CURRENT_DATE() - INTERVAL 1 YEAR)
  ) * 0.5
GROUP BY u.id, u.full_name, u.email
```

This query joins the `users`, `orders`, `order_items`, and `products` tables. It filters the orders based on the current month and year u